# 7장, 데이터 정리 (cleaning)

https://wesmckinney.com/book/data-cleaning

In [2]:
!pip install -q numpy pandas

In [3]:
!pip install -Uq nbdev

In [2]:
import numpy as np
import pandas as pd

## 7.1 누락된 데이터 처리

누락된 데이터 필터링
누락된 데이터 채우기



## 7.2 데이터 변환

### 중복 제거
https://wesmckinney.com/book/data-cleaning#prep_clean_deduplicate

In [11]:
data = pd.DataFrame({"k1": ["one", "two"] * 3 + ["two"],
                     "k2": [1, 1, 2, 3, 3, 4, 4]})
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


In [ ]:
# 각 행이 중복인지(해당 열 값이 "이전 행의 열 값" 과 "정확히" 같은지) 여부의 bool Series
_ = data.duplicated()
print(_, type(_), _.shape)

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool <class 'pandas.core.series.Series'> (7,)


In [ ]:
data.drop_duplicates()
# 순서는 지켜지겠으나, 인덱스는 더 이상 id 역할을 하지 못한다.
# 이 함수는 사본을 리턴하므로, 원본은 변하지 않는다.

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


In [21]:
data['v1'] = range(7)  # 컬럼을 추가하는 간단한 방법
print(data)

    k1  k2  v1
0  one   1   0
1  two   1   1
2  one   2   2
3  two   3   3
4  one   3   4
5  two   4   5
6  two   4   6


In [23]:
data.drop_duplicates(subset=['k1'])  # k1 만 비교해서 중복 제거

,k1,k2,v1
0,one,1,0
1,two,1,1


In [ ]:
data.drop_duplicates(["k1", "k2"], keep="last") # 중복 행 들 중 마지막 행을 유지.

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
6,two,4,6


### 함수 또는 매핑을 사용하여 데이터 변환
https://wesmckinney.com/book/data-cleaning#prep_mapping_values

```
map(arg: ((Any) -> (S2@map | NAType)) |
         Mapping[Any, S2@map] |
         Series[S2@map], ...
    )
```

많은 데이터 세트에서 배열, 시리즈 또는 DataFrame의 열 값을 기반으로 변환을 수행해야 할 수 있다. <br>

아래는 다양한 종류의 육류에 대해 수집된 가상 데이터 이다.

In [3]:
data = pd.DataFrame({"food": ["bacon", "pulled pork", "bacon",
                     "pastrami", "corned beef", "bacon",
                     "pastrami", "honey ham", "nova lox"],
            "ounces": [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,pastrami,6.0
4,corned beef,7.5
5,bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


In [4]:
# 각 식품의 동물 종류를 나타내는 열을 추가하고 싶다고 가정 해보자. 각 고기 종류를 동물 종류에 매핑한다.
meat_to_animal = {
  "bacon": "pig",
  "pulled pork": "pig",
  "pastrami": "cow",
  "corned beef": "cow",
  "honey ham": "pig",
  "nova lox": "salmon"
}

In [ ]:
# Series 의 map 메서드(5.2.5장: 함수 적용 및 매핑에서도 설명) 는 값의 변환을 수행하기 위한 매핑을 포함하는 함수나 사전과 같은 객체를 허용한다.
# data 의 한 열에 선택한 후 .map() 호출한 것이므로, Series 의 .map() 이다.
#
data["animal"] = data["food"].map(meat_to_animal)
    #
    # 여기서 .map() 메소드는 Series 의 메소드 이다.
    # 지금 사용 예시는 사전(dictionary)를 인자로 전달한 것이다.
    # 결과로 새로운 'animal' 컬럼이 추가된다.

data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,pastrami,6.0,cow
4,corned beef,7.5,cow
5,bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


In [6]:
# .map() 에 '사전' 이 아니라 함수를 전달할 수도 있다.
def get_animal(x):
    return meat_to_animal[x]

data["food"].map(get_animal)
# map() 메소드는 새로운 series 객체를 리턴하므로, 원본 자체는 변경시키지 않는다.

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

### 값 바꾸기

- `map()`
- `replace(to_replace: Any = ..., value: Any = ...,)`

`fillna` 메서드를 사용하여 누락된 데이터를 채우는 것은 보다 일반적인 값 대체의 특별한 경우입니다.<br>
이미 살펴보았듯이, `map` 메서드는 객체의 값 하위 집합을 수정하는 데 사용할 수 있지만, `replace` 메서드는 더 간단하고 유연한 방법을 제공합니다.

In [ ]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

In [ ]:
# -999 값은 누락된 데이터에 대한 센티널 값일 수 있습니다.
# 이 값을 판다스가 인식하는 NA 값으로 대체하려면 replace 함수를 사용하여 새로운 시리즈를 생성할 수 있습니다.

data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

In [ ]:
# 여러 값을 한번에 바꾸기. 두 개의 리스트를 전달.

data.replace([-999, -1000], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

### 축 인덱스 이름 바꾸기
https://wesmckinney.com/book/data-cleaning#prep_renaming

- `df.index.map()`
- `df.rename(index=Renamer, ..)`

Series의 값과 마찬가지로, 축 레이블도 함수나 특정 형태의 매핑을 통해 변형되어 레이블이 다르게 지정된 새로운 객체를 생성할 수 있습니다. <br>
또한 새로운 데이터 구조를 생성하지 않고도 축을 수정할 수 있습니다.


In [7]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
                   index=["Ohio", "Colorado", "New York"],
                   columns=["one", "two", "three", "four"])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [8]:
print(type(data.index))
# Index 라는 클래스는 Series 와는 다르긴 하지만 여러모로 Series 와 비슷한 거 같다.
data.index


<class 'pandas.core.indexes.base.Index'>


Index(['Ohio', 'Colorado', 'New York'], dtype='object')

In [9]:
def transform(x):
    return x[:4].upper() # 앞 네 글자 추출, 대문자화

data.index.map(transform) # Series 에서 했던 것 처럼, Index에 대해서도 매핑 변환을 할 수 있다.
# Index(['OHIO', 'COLO', 'NEW '], dtype='object')

Index(['OHIO', 'COLO', 'NEW '], dtype='object')

In [10]:
# DataFrame 의 인덱스를, 방금 전 변환한 새 인덱스로 "교체" 할 수 있음.
data.index = data.index.map(transform)
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


원본을 수정하지 않고 변형된 버전의 데이터 세트를 만들려면 rename 메소드를 사용하는 것이 좋습니다.

In [11]:
data.rename(index=str.title, columns=str.upper)
# 이 rename() 메소드는 새로운 DataFrame 객체를 리턴한다.
# str.title, str.upper 는 무엇인가? str 은 무엇을 가리키나?

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colo,4,5,6,7
New,8,9,10,11


In [12]:
# 축의 label 전체가 아닌 일부 만 새 label 로 변경할 수도 있다. 사전-like 한 객체를 전달하면 된다.
data.rename(index={"OHIO": "INDIANA"},
           columns={"three": "peekaboo"})

# rename()을 사용하면 다음 두 과정을 한번에 수행한다.
#   - DataFrame을 수동으로 복사
#   - 인덱스 및 열 속성에 새 값을 할당

,one,two,peekaboo,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


### 이산화 및 비닝
### 이상치 감지 및 필터링

### 순열 및 무작위 샘플링


In [13]:
df = pd.DataFrame(np.arange(5 * 7).reshape((5, 7)))
df

,0,1,2,3,4,5,6
0,0,1,2,3,4,5,6
1,7,8,9,10,11,12,13
2,14,15,16,17,18,19,20
3,21,22,23,24,25,26,27
4,28,29,30,31,32,33,34


### 지표/더미 변수 계산



## 7.3 확장 데이터 유형

- 2025/4/24

- pandas는 원래 숫자형 데이터 작업에 주로 사용되는 배열 컴퓨팅 라이브러리인 NumPy의 기능을 기반으로 구축되었음.
  - 누락된 데이터와 같은 많은 pandas 개념은 NumPy 에서 사용 가능한 기능을 사용하여 구현되었음.
  - NumPy 와 pandas 를 함께 사용하는 라이브러리 간의 호환성을 극대화하려고 노력.

- NumPy를 기반으로 구축되면서 다음과 같은 여러 단점이 발생.
  - 정수 및 부울과 같은 일부 숫자형 데이터 유형에 대한 누락된 데이터 처리가 불완전
    - 결과적으로 이러한 데이터에 누락된 데이터가 추가되면, pandas 는 해당 데이터 유형을 float64로 변환하고 np.nan 을 사용하여 null 값을 나타냄.
    - 이는 많은 pandas 알고리즘에 미묘한 문제를 야기하여 복합적인 효과를 냄
  - 문자열 데이터가 많은 데이터 세트는 계산 비용이 많이 들고 메모리를 많이 사용.
  - 시간 간격, 타임델타, 시간대가 포함된 타임스탬프와 같은 일부 데이터 유형은 계산 비용이 많이 드는 Python 객체 배열을 사용해야 했음.

- 최근 Pandas 는 NumPy 에서 기본적으로 지원하지 않는 데이터 유형도 추가할 수 있는 확장 유형 시스템을 개발함.
  - 이러한 새로운 데이터 유형은 NumPy 배열에서 들어오는 데이터와 함께 일급 데이터로 처리될 수 있음.


In [ ]:
s = pd.Series([1, 2, 3, ])
print(s.dtype) # int64
s

int64


0    1
1    2
2    3
dtype: int64

In [ ]:
s = pd.Series([1, 2, 3, None])
print(s.dtype) # float64
# 단순하게 None 이 하나 추가되었을 뿐인데 이 시리즈 전체가 float 형으로 바뀐다.
# float 는 NaN 을 공식적으로 표현할 수 있기 때문. IEEE 754 참고.
s

float64


0    1.0
1    2.0
2    3.0
3    NaN
dtype: float64

In [ ]:
s = pd.Series([1, 2, 3, None], dtype=pd.Int64Dtype())
s
# 처음에는 int64 였는데, 여기서는 Int64 로 표시됨. 대문자, 소문자 차이가 나는데, 서로 다른 타입이다.
# Int64 는 nullable integer 타입으로서, 내부에 NA 여부를 저장하는 별도의 마스크 정보가 있다.

print(s.to_numpy().dtype)
s.to_numpy()
# numpy 로 변환하면 다시 np.nan 으로 표현해야 하기 때문에, dtype 이 float 형으로 바뀐다.
#
# 참고: NaN (Not a Number) 와 NA 는 엄밀하게 말하면 그 의미가 다르다. np에서는 NA 표현을 위해 nan을 빌려다 쓰는 것임.

float64


array([ 1.,  2.,  3., nan])

In [ ]:
print(s.dtype) # Int64
print(type(s.dtype))
    # print 로 표시할 때는 "Int64" 였는데, 실제 타입 이름은 Int64Dtype() 이다.

print(s.isna())  # False False False True

print(s[3])  # <NA>
print(s[3] is pd.NA) # True

Int64
<class 'pandas.core.arrays.integer.Int64Dtype'>
0    False
1    False
2    False
3     True
dtype: bool
<NA>
True


In [ ]:
# dtype 에 클래스 (타입) 을 지정해도 되지만, 좀 더 간편하게 문자열로 된 타입 이름을 쓸 수도 있다.
#t
# s = pd.Series([1, 2, 3, None], dtype=pd.Int64Dtype()) # 앞에서는 이렇게 했는데..
s = pd.Series([1, 2, 3, None], dtype="Int64")  # 이렇게 할 수도 있다. 이 둘은 100% 동일하다.
s

0       1
1       2
2       3
3    <NA>
dtype: Int64

In [ ]:
# numpy 에서 문자열은 그냥 object 였지만, pandas 에서는 특별히 string 타입을 만들었다.
# 이 타입이 좀 더 효율적이고, 메모리도 적게 소모한다고 함.

s = pd.Series(['one', 'two', None, 'three'], dtype=pd.StringDtype())
s

0      one
1      two
2     <NA>
3    three
dtype: string

### astype

In [25]:
df = pd.DataFrame({"A": [1, 2, None, 4],
                   "B": ["one", "two", "three", None],
                   "C": [False, None, False, True] })
df

,A,B,C
0,1.0,one,False
1,2.0,two,None
2,NaN,three,False
3,4.0,None,True


In [27]:
df["A"] = df["A"].astype("Int64")
df["B"] = df["B"].astype("string")
df["C"] = df["C"].astype("boolean")
df

,A,B,C
0,1,one,False
1,2,two,<NA>
2,<NA>,three,False
3,4,<NA>,True


끝

## 7.4 문자열 조작

Python 내장 문자열 객체 메서드
정규 표현식
판다스의 문자열 함수



## 7.5 범주형 데이터

- 2025/4/15 검토.

### 배경 및 동기

In [28]:
fruits = pd.Series(['apple', 'orange', 'apple', 'apple'] * 2)
fruits

# 숫자가 아니라면 타입은 모두 object 이다.  dtype=object

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
dtype: object

In [ ]:
fu = pd.unique(fruits)
print(type(fu))
fu
# numpy 에도 unique() 가 있지만, pd.unique()가 더 빠르고 N/A 처리도 깔끔하다고 함.
#  이 예제에 관해서는 두 방식의 결과는 완전히 동일하다.
#  둘 다 ndarray 타입의 리스트를 리턴하고, dtype 은 object 이다.
# np.unique(fruits)

<class 'numpy.ndarray'>


array(['apple', 'orange'], dtype=object)

In [34]:
#pd.value_counts(fruits) # deprecated
fruits.value_counts()
# Series 를 리턴하다. 원래 데이터의 값이 이번에는 인덱스로 간주된다.

apple     6
orange    2
Name: count, dtype: int64

### 판다스의 범주형 확장 유형

In [65]:
df = pd.DataFrame({'fruit':['apple', 'orange'], 'count':[20,30]})
print(df.fruit.array)
df.fruit.__dict__.keys()

<NumpyExtensionArray>
['apple', 'orange']
Length: 2, dtype: object


dict_keys(['_is_copy', '_mgr', '_item_cache', '_attrs', '_flags', '_name', '_cacher'])

In [61]:
import inspect
print(inspect.getsource(pd.Series.array.fget))

# pd.api.extensions.ExtensionArray?

    @Appender(base.IndexOpsMixin.array.__doc__)  # type: ignore[misc]
    @property
    def array(self) -> ExtensionArray:
        return self._mgr.array_values()



In [36]:
N = len(fruits)
print(N)

rng = np.random.default_rng(seed=12345)
print(rng)

df = pd.DataFrame({'fruit': fruits,
                   'basket_id': np.arange(N),
                   'count': rng.integers(3, 15, size=N),
                   'weight': rng.uniform(0, 4, size=N)},
                  columns=['basket_id', 'fruit', 'count', 'weight'])
df

8
Generator(PCG64)


,basket_id,fruit,count,weight
0,0,apple,11,1.564438
1,1,orange,5,1.331256
2,2,apple,12,2.393235
3,3,apple,6,0.746937
4,4,apple,5,2.691024
5,5,orange,12,3.767211
6,6,apple,10,0.992983
7,7,apple,11,3.795525


In [37]:
print(df['fruit'])

# 이 Series 의 타입을 굳이 얘기하자면 여전히 object 이다.
# Name: fruit, dtype: object

print(df.fruit.dtype) # object
df['fruit'].dtype # dtype('O')  # 아마도 Object 를 의미하는 게 아닐까..

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: object
object


dtype('O')

In [38]:
df['fruit'].describe()

count         8
unique        2
top       apple
freq          6
Name: fruit, dtype: object

In [ ]:
print(type(df.fruit))


# print(type(df['fruit'].array)) # ... NumpyExtensionArray
print(type(df.fruit.array)) # ... NumpyExtensionArray

print(df.fruit.__dict__.keys())
# dict_keys(['_is_copy', '_mgr', '_item_cache', '_attrs', '_flags', '_name', '_cacher', 'str'])
# 사실 df.fruit 에는 array 라는 속성은 없다. 부모 클래스의 속성인 것으로 보인다.

df.fruit.array

<class 'pandas.core.series.Series'>
<class 'pandas.core.arrays.numpy_.NumpyExtensionArray'>
dict_keys(['_is_copy', '_mgr', '_item_cache', '_attrs', '_flags', '_name', '_cacher'])
fruit


<NumpyExtensionArray>
['apple', 'orange']
Length: 2, dtype: object

In [21]:
# astype() 은 지정한 타입으로 변환하는 일종의 캐스팅 이다.

fruit_cat = df['fruit'].astype('category')

print(type(fruit_cat)) # pd.Series
fruit_cat
# 맨 아래 라인에, 타입을 보여준다. 즉, 타입이 바뀌었음을 확인할 수 있다.
# Name: fruit, dtype: category

<class 'pandas.core.series.Series'>


0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): ['apple', 'orange']

In [22]:
c = fruit_cat.array
print(type(c))  # pandas.core.arrays.categorical.Categorical
c

# 원래는 NumpyExtension 뭐 이런 클래스였는데, Categorical 로 바뀌었음.

<class 'pandas.core.arrays.categorical.Categorical'>


['apple', 'orange', 'apple', 'apple', 'apple', 'orange', 'apple', 'apple']
Categories (2, object): ['apple', 'orange']

In [23]:
# 두 개의 중요한 속성: categories 와 codes

print(type(c.categories))  # pandas.core.indexes.base.Index
c.categories

<class 'pandas.core.indexes.base.Index'>


Index(['apple', 'orange'], dtype='object')

In [24]:
print(type(c.codes))  # np.ndarray, dtype=int?
c.codes

<class 'numpy.ndarray'>


array([0, 1, 0, 0, 0, 1, 0, 0], dtype=int8)

In [25]:
# 코드와 범주 간의 매핑

dict(enumerate(c.categories))

{0: 'apple', 1: 'orange'}

In [26]:
# 이제 특정 열을, 지금까지 알아본 범주형으로 전환할 수 있다.
df['fruit'] = df['fruit'].astype('category')
df

# 사실 표시되는 내용 만으로는 뭐가 달라졌는지 눈치채긴 어렵다.
# 범주형으로 바뀌어도 표시할 때 code 로 표시하는 것은 아니니까..

,basket_id,fruit,count,weight
0,0,apple,11,1.564438
1,1,orange,5,1.331256
2,2,apple,12,2.393235
3,3,apple,6,0.746937
4,4,apple,5,2.691024
5,5,orange,12,3.767211
6,6,apple,10,0.992983
7,7,apple,11,3.795525


In [27]:
df.fruit
# 이렇게 직접 컬럼을 표시해 봐야 타입이 범주형 인지를 알 수 있다.  (dtype: category)

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): ['apple', 'orange']

In [28]:
# 범주형 인코딩의 여러 방법들.

# 1. 직접 Categorical 생성자를 사용.
my_categories = pd.Categorical(['foo', 'bar', 'baz', 'foo', 'bar'])
print(my_categories)
print(my_categories.codes) # 이 코드는 내부적으로 결정됨.

['foo', 'bar', 'baz', 'foo', 'bar']
Categories (3, object): ['bar', 'baz', 'foo']
[2 0 1 2 0]


In [29]:
# 코드 할당에 특정한 순서가 있는가??? 알파벳 순서 같기도 하고..
dict(enumerate(my_categories.categories)) # {0: 'bar', 1: 'baz', 2: 'foo'}

{0: 'bar', 1: 'baz', 2: 'foo'}

In [30]:
categories = ['foo', 'bar', 'baz']

codes = [0, 1, 2, 0, 0, 1]

# 이렇게 직접 categories 를 지정하면 리스트 순서대로 코드가 부여되고 있음.
my_cats_2 = pd.Categorical.from_codes(codes, categories)
my_cats_2

['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo', 'bar', 'baz']

In [31]:
dict(enumerate(my_cats_2.categories))  # {0: 'foo', 1: 'bar', 2: 'baz'}

{0: 'foo', 1: 'bar', 2: 'baz'}

In [32]:
# 순서에 따라 바뀌나??
codes = [1, 2, 0, 1, 1, 2]

my_cats_2 = pd.Categorical.from_codes(codes, categories)
my_cats_2

['bar', 'baz', 'foo', 'bar', 'bar', 'baz']
Categories (3, object): ['foo', 'bar', 'baz']

In [33]:
ordered_cat = pd.Categorical.from_codes(codes, categories, ordered=True)
ordered_cat
# 표시 되는 형태로부터 이게 ordered 임을 알 수 있다.
# Categories (3, object): ['foo' < 'bar' < 'baz']

['bar', 'baz', 'foo', 'bar', 'bar', 'baz']
Categories (3, object): ['foo' < 'bar' < 'baz']

In [34]:
# 순서 없는 인스턴스를 순서 있는 타입으로..
my_cats_2.as_ordered()

['bar', 'baz', 'foo', 'bar', 'bar', 'baz']
Categories (3, object): ['foo' < 'bar' < 'baz']

### Categorical 를 사용한 계산


In [35]:
rng = np.random.default_rng(seed=12345)

draws = rng.standard_normal(1000)

draws[:5]
# array([-1.4238,  1.2637, -0.8707, -0.2592, -0.0753])

array([-1.42382504,  1.26372846, -0.87066174, -0.25917323, -0.07534331])

In [36]:
# 비닝
bins = pd.qcut(draws, 4)
# 모든 요소를 4개의 범주로 구분한다. 여기서 범주 (인덱스)는 숫자의 범위가 된다.
# 인덱스가 출력 용으로는 그다지 적합하지 않다.
bins


[(-3.121, -0.675], (0.687, 3.211], (-3.121, -0.675], (-0.675, 0.0134], (-0.675, 0.0134], ..., (0.0134, 0.687], (0.0134, 0.687], (-0.675, 0.0134], (0.0134, 0.687], (-0.675, 0.0134]]
Length: 1000
Categories (4, interval[float64, right]): [(-3.121, -0.675] < (-0.675, 0.0134] < (0.0134, 0.687] < (0.687, 3.211]]

In [37]:
# 그래서 레이블을 지정하는 것이 좋다.
bins = pd.qcut(draws, 4, labels=['Q1', 'Q2', 'Q3', 'Q4'])
bins

['Q1', 'Q4', 'Q1', 'Q2', 'Q2', ..., 'Q3', 'Q3', 'Q2', 'Q3', 'Q2']
Length: 1000
Categories (4, object): ['Q1' < 'Q2' < 'Q3' < 'Q4']

In [38]:
bins.codes[:10]  # 다 표시하면 1000개나 되니까 일부만 출력해 보자.

# codes 는 대부분 dtype=int8 인 것 같다. 범주 수가 127 을 넘지 않으면 그렇게 되는 듯.

array([0, 3, 0, 1, 1, 0, 0, 2, 2, 0], dtype=int8)

In [39]:
# 128 이상으로 만들어 볼까?
c = pd.Categorical([ f'a{i:03}' for i in range(1,130) ])
c.codes[:5], c.codes[-5:]

# 역시나 codes 의 dtype 이 int16 이 되었음.


(array([0, 1, 2, 3, 4], dtype=int16),
 array([124, 125, 126, 127, 128], dtype=int16))

In [40]:
bins = pd.Series(bins, name='quartile')

# 아래 코드는 솔직히 말해 잘 모르기도 하고, 이해하는데 시간이 많이 들 거 같아서 패스!
results = (pd.Series(draws)
            .groupby(bins)
            .agg(['count', 'min', 'max'])
            .reset_index())

/var/folders/gk/5yll9c4s4574ff6wr892k56r0000gn/T/ipykernel_50005/1689744025.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  results = (pd.Series(draws)


In [41]:
pd.Series(draws).groupby(bins)

/var/folders/gk/5yll9c4s4574ff6wr892k56r0000gn/T/ipykernel_50005/3698549966.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  pd.Series(draws).groupby(bins)


### Categorical 을 이용하여 성능 향상

In [42]:
# 요약하자면..
#  1회성 변환 (object -> categorical) 비용을 들여서, 그 이후의 많은 계산에서의 성능의 혜택을 본다는 것임.
#  메모리 사용량 또한 대폭 줄어듬. (당연한 얘기지만..)


### 범주형 메소드

In [43]:
s = pd.Series(['a', 'c', 'd', 'b'] * 2)

cat_s = s.astype('category')
print(type(cat_s))  # pd.Series
cat_s               # dtype: category

<class 'pandas.core.series.Series'>


0    a
1    c
2    d
3    b
4    a
5    c
6    d
7    b
dtype: category
Categories (4, object): ['a', 'b', 'c', 'd']

In [44]:
#  # pandas.core.arrays.categorical.Categorical
print(type(cat_s.array))
cat_s.array


<class 'pandas.core.arrays.categorical.Categorical'>


['a', 'c', 'd', 'b', 'a', 'c', 'd', 'b']
Categories (4, object): ['a', 'b', 'c', 'd']

In [45]:
# 접근자.
print(type(cat_s.cat))
cat_s.cat

# .array 를 이용해 내부의 Categorical 에 직접 접근할 수 있는데도
# 굳이 이렇게 .cat 이라는 Accessor 가 필요한 이유는???

<class 'pandas.core.arrays.categorical.CategoricalAccessor'>


In [46]:
# .cat:  접근자 (accessor) 프로퍼티. 이것을 통해 여러 메소드에 접근 가능.
#
print(type(cat_s.cat.codes))  # Series
cat_s.cat.codes[:3]  # .code 는 메소드는 아니고 프로퍼티 이지만.

<class 'pandas.core.series.Series'>


0    0
1    2
2    3
dtype: int8

In [47]:
print(type(cat_s.cat.categories)) # Index
cat_s.cat.categories # 이것의 타입은 Index

<class 'pandas.core.indexes.base.Index'>


Index(['a', 'b', 'c', 'd'], dtype='object')

#### 모델링을 위한 더미 변수 생성

In [48]:
cat_s = pd.Series(['a', 'b', 'c', 'd'] * 2, dtype='category')
cat_s

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (4, object): ['a', 'b', 'c', 'd']

In [49]:
# 원-핫 인코딩을 위한 더미 변수로 변환. 이때 범주의 이름이 컬럼 이름이 됨.
pd.get_dummies(cat_s, dtype=int)

,a,b,c,d
0,1,0,0,0
1,0,1,0,0
2,0,0,1,0
3,0,0,0,1
4,1,0,0,0
5,0,1,0,0
6,0,0,1,0
7,0,0,0,1
